In [1]:
# Install required packages
!pip install easyocr opencv-python-headless pillow fuzzywuzzy python-levenshtein
!apt-get update
!apt-get install -y tesseract-ocr
!pip install pytesseract

Defaulting to user installation because normal site-packages is not writeable


'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt-get' is not recognized as an internal or external command,
operable program or batch file.


Defaulting to user installation because normal site-packages is not writeable


In [2]:
#!/usr/bin/env python3
import sqlite3
import cv2
import numpy as np
import re
import os
import sys
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
from difflib import SequenceMatcher
import warnings
warnings.filterwarnings('ignore')

# Installation hint for Kaggle
# KAGGLE_INSTALL_COMMANDS = """
# !pip install easyocr opencv-python-headless pillow fuzzywuzzy python-levenshtein
# !apt-get update && apt-get install -y tesseract-ocr
# !pip install pytesseract
# """

# print("=" * 60)
# print("PRESCRIPTION MEDICINE EXTRACTION PIPELINE")
# print("=" * 60)
# print("\nFor Kaggle setup, run these commands first:")
# print(KAGGLE_INSTALL_COMMANDS)
# print("=" * 60)


In [3]:
try:
    import easyocr
    import pytesseract
    from PIL import Image, ImageDraw, ImageFont
    from fuzzywuzzy import fuzz, process
    print("✓ All required libraries imported successfully")
except ImportError as e:
    print(f"⚠ Missing library: {e}")


✓ All required libraries imported successfully


In [4]:
@dataclass
class MedicineInfo:
    """Data class for medicine information"""
    name: str
    description: str
    uses: str
    dosage: str
    confidence: float = 0.0


In [5]:
class PrescriptionDatabase:
    def __init__(self, db_path: str = "medicines.db"):
        self.db_path = db_path
        self.conn = None
        self.create_database()
    
    def create_database(self):
        self.conn = sqlite3.connect(self.db_path)
        cursor = self.conn.cursor()
        
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS medicines (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            medicine_name TEXT NOT NULL UNIQUE,
            description TEXT NOT NULL,
            uses TEXT NOT NULL,
            recommended_dosage TEXT NOT NULL
        )
        ''')

        medicines_data = [
            ("Paracetamol", "Analgesic and antipyretic", "Fever, headache, mild pain", "500mg every 6 hours"),
            ("Ibuprofen", "Non-steroidal anti-inflammatory drug", "Pain, inflammation, fever", "400mg every 8 hours"),
            ("Aspirin", "Antiplatelet and analgesic", "Heart protection, pain relief", "75mg once daily"),
            ("Amoxicillin", "Penicillin antibiotic", "Bacterial infections", "500mg every 8 hours"),
            ("Omeprazole", "Proton pump inhibitor", "Acid reflux, stomach ulcers", "20mg once daily"),
            ("Metformin", "Antidiabetic medication", "Type 2 diabetes", "500mg twice daily"),
            ("Lisinopril", "ACE inhibitor", "High blood pressure, heart failure", "10mg once daily"),
            ("Atorvastatin", "Statin medication", "High cholesterol", "20mg once daily"),
            ("Amlodipine", "Calcium channel blocker", "High blood pressure", "5mg once daily"),
            ("Simvastatin", "Statin medication", "High cholesterol", "40mg once daily"),
            ("Losartan", "Angiotensin receptor blocker", "High blood pressure", "50mg once daily"),
            ("Gabapentin", "Anticonvulsant", "Nerve pain, seizures", "300mg three times daily"),
            ("Prednisone", "Corticosteroid", "Inflammation, autoimmune conditions", "5-10mg daily"),
            ("Sertraline", "SSRI antidepressant", "Depression, anxiety", "50mg once daily"),
            ("Fluoxetine", "SSRI antidepressant", "Depression, anxiety", "20mg once daily"),
            ("Lorazepam", "Benzodiazepine", "Anxiety, insomnia", "1mg as needed"),
            ("Tramadol", "Opioid analgesic", "Moderate to severe pain", "50mg every 6 hours"),
            ("Hydrochlorothiazide", "Thiazide diuretic", "High blood pressure, fluid retention", "25mg once daily"),
            ("Warfarin", "Anticoagulant", "Blood clot prevention", "5mg daily (adjusted)"),
            ("Digoxin", "Cardiac glycoside", "Heart failure, atrial fibrillation", "0.25mg once daily"),
            ("Furosemide", "Loop diuretic", "Fluid retention, heart failure", "40mg once daily"),
            ("Pantoprazole", "Proton pump inhibitor", "Acid reflux, stomach protection", "40mg once daily"),
            ("Ciprofloxacin", "Fluoroquinolone antibiotic", "Bacterial infections", "500mg twice daily"),
            ("Azithromycin", "Macrolide antibiotic", "Bacterial infections", "500mg once daily"),
            ("Cephalexin", "Cephalosporin antibiotic", "Bacterial infections", "500mg four times daily"),
            ("Levothyroxine", "Thyroid hormone", "Hypothyroidism", "75mcg once daily"),
            ("Albuterol", "Bronchodilator", "Asthma, COPD", "2 puffs every 4 hours"),
            ("Montelukast", "Leukotriene receptor antagonist", "Asthma, allergies", "10mg once daily"),
            ("Cetirizine", "Antihistamine", "Allergies, hay fever", "10mg once daily"),
            ("Loratadine", "Antihistamine", "Allergies, hay fever", "10mg once daily"),
            ("Ranitidine", "H2 receptor antagonist", "Acid reflux, stomach ulcers", "150mg twice daily"),
            ("Clonazepam", "Benzodiazepine", "Anxiety, seizures", "0.5mg twice daily"),
            ("Hydrocodone", "Opioid analgesic", "Moderate to severe pain", "5mg every 6 hours"),
            ("Metoprolol", "Beta-blocker", "High blood pressure, heart conditions", "50mg twice daily"),
            ("Carvedilol", "Beta-blocker", "Heart failure, high blood pressure", "6.25mg twice daily"),
        ]
        
        cursor.executemany('''
        INSERT OR REPLACE INTO medicines 
        (medicine_name, description, uses, recommended_dosage) 
        VALUES (?, ?, ?, ?)
        ''', medicines_data)
        
        self.conn.commit()
        print(f"✓ Database created with {len(medicines_data)} medicines")
    
    def search_medicine(self, medicine_name: str) -> Optional[MedicineInfo]:
        cursor = self.conn.cursor()
        cursor.execute('''
        SELECT medicine_name, description, uses, recommended_dosage 
        FROM medicines 
        WHERE LOWER(medicine_name) LIKE LOWER(?)
        ''', (f'%{medicine_name}%',))
        result = cursor.fetchone()
        return MedicineInfo(*result) if result else None
    
    def get_all_medicine_names(self) -> List[str]:
        cursor = self.conn.cursor()
        cursor.execute('SELECT medicine_name FROM medicines')
        return [row[0] for row in cursor.fetchall()]
    
    def close(self):
        if self.conn:
            self.conn.close()


In [6]:
class OCRProcessor:
    def __init__(self):
        try:
            self.easy_reader = easyocr.Reader(['en'])
            self.tesseract_available = True
            print("✓ OCR engines initialized")
        except Exception as e:
            print(f"⚠ OCR init warning: {e}")
            self.easy_reader = None
            self.tesseract_available = False
    
    def extract_text_easyocr(self, image_path: str) -> List[Tuple[str, float]]:
        if not self.easy_reader:
            return []
        try:
            results = self.easy_reader.readtext(image_path)
            return [(text, conf) for _, text, conf in results]
        except Exception as e:
            print(f"EasyOCR error: {e}")
            return []
    
    def extract_text_tesseract(self, image_path: str) -> List[Tuple[str, float]]:
        if not self.tesseract_available:
            return []
        try:
            img = cv2.imread(image_path)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            processed = self.preprocess_image(gray)
            pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

            data = pytesseract.image_to_data(processed, output_type=pytesseract.Output.DICT)
            return [
                (data['text'][i].strip(), int(data['conf'][i]) / 100.0)
                for i in range(len(data['text']))
                if data['text'][i].strip() and int(data['conf'][i]) > 10
            ]
        except Exception as e:
            print(f"Tesseract error: {e}")
            return []

    def preprocess_image(self, image):
        denoised = cv2.fastNlMeansDenoising(image)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        contrast = clahe.apply(denoised)
        _, thresh = cv2.threshold(contrast, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        return thresh


In [15]:
class MedicineExtractor:
    def __init__(self):
        self.db = PrescriptionDatabase()
        self.ocr = OCRProcessor()
        self.medicine_names = self.db.get_all_medicine_names()
        self.medicine_patterns = [
            r'\b[A-Z][a-z]+(?:olol|pril|sartan|statin|cillin|mycin|oxacin)\b',
            r'\b[A-Z][a-z]{3,}\b',
            r'\b\d+\s*mg\b',
        ]

    def extract_medicines_from_image(self, image_path: str) -> List[MedicineInfo]:
        print(f"\n🔍 Processing image: {image_path}")
        easyocr_results = self.ocr.extract_text_easyocr(image_path)
        tesseract_results = self.ocr.extract_text_tesseract(image_path)
        all_texts = easyocr_results + tesseract_results
        print(f"✓ Extracted {len(all_texts)} text segments")
        potential_medicines = self.extract_potential_medicines(all_texts)
        print(f"✓ Found {len(potential_medicines)} potential medicines")
        matched_medicines = self.match_medicines_fuzzy(potential_medicines)
        print(f"✓ Matched {len(matched_medicines)} medicines in database")
        return matched_medicines

    # def extract_potential_medicines(self, text_results: List[Tuple[str, float]]) -> List[Tuple[str, float]]:
    #     potential_medicines = []
    #     for text, confidence in text_results:
    #         cleaned_text = re.sub(r'[^\w\s]', '', text)
    #         words = cleaned_text.split()
    #         for word in words:
    #             if len(word) > 3:
    #                 for pattern in self.medicine_patterns:
    #                     if re.search(pattern, word, re.IGNORECASE):
    #                         potential_medicines.append((word, confidence))
    #                         break
    #                 else:
    #                     if len(word) > 5:
    #                         potential_medicines.append((word, confidence))
    #     return potential_medicines

    def extract_potential_medicines(self, text_results):
        joined_lines = self._merge_boxes_into_lines(text_results)
        
        potential = []
        for line, conf in joined_lines:
            # Strip dosage part (everything after the first number)
            line = re.split(r'\d', line, 1)[0]
            # Take the longest capitalised token(s)
            tokens = [w for w in re.findall(r'[A-Za-z]+', line) if w.istitle()]
            guess = " ".join(tokens[:2])          # handles multi‑word brands
            if len(guess) >= 4:
                potential.append((guess, conf))
        return potential


    def match_medicines_fuzzy(self, potential_medicines: List[Tuple[str, float]]) -> List[MedicineInfo]:
        matched_medicines = []
        for medicine_text, ocr_confidence in potential_medicines:
            medicine_info = self.db.search_medicine(medicine_text)
            if medicine_info:
                medicine_info.confidence = ocr_confidence
                matched_medicines.append(medicine_info)
            else:
                best_match = process.extractOne(
                    medicine_text, self.medicine_names, scorer=fuzz.token_sort_ratio, score_cutoff=60
                )
                if best_match:
                    matched_name, similarity_score = best_match
                    medicine_info = self.db.search_medicine(matched_name)
                    if medicine_info:
                        combined_confidence = (ocr_confidence + similarity_score/100) / 2
                        medicine_info.confidence = combined_confidence
                        matched_medicines.append(medicine_info)

        unique_medicines = {}
        for medicine in matched_medicines:
            if medicine.name not in unique_medicines or \
               medicine.confidence > unique_medicines[medicine.name].confidence:
                unique_medicines[medicine.name] = medicine
        return list(unique_medicines.values())

    def _merge_boxes_into_lines(self, text_results):
        """
        Takes OCR results in (text, confidence) form and merges lines
        that appear to be on the same line in the image.
        Since we're simulating, we just return the text as-is for now.
        """
        # If text_results is a list of tuples like: [(text1, conf1), (text2, conf2), ...]
        # You can just return it for now, or later add logic to merge similar lines
        return text_results



In [8]:
def create_sample_prescription(output_path: str = "sample_prescription.png"):
    try:
        width, height = 800, 600
        img = Image.new('RGB', (width, height), color='white')
        draw = ImageDraw.Draw(img)

        try:
            font = ImageFont.truetype("arial.ttf", 24)
            small_font = ImageFont.truetype("arial.ttf", 18)
        except:
            font = ImageFont.load_default()
            small_font = ImageFont.load_default()

        draw.text((50, 50), "Dr. John Smith", fill='black', font=font)
        draw.text((50, 80), "Medical Prescription", fill='black', font=font)
        draw.text((50, 110), "Date: 2024-01-15", fill='black', font=small_font)
        draw.line([(50, 140), (750, 140)], fill='black', width=2)

        medicines = [
            "1. Paracetamol 500mg - Take twice daily",
            "2. Ibuprofen 400mg - Take every 8 hours",
            "3. Omeprazole 20mg - Once daily before breakfast",
            "4. Amoxicillin 500mg - Three times daily",
            "5. Metformin 500mg - Twice daily with meals"
        ]

        y_pos = 180
        for medicine in medicines:
            draw.text((70, y_pos), medicine, fill='black', font=small_font)
            y_pos += 40

        draw.line([(50, height-100), (750, height-100)], fill='black', width=1)
        draw.text((50, height-80), "Doctor's Signature", fill='black', font=small_font)
        img.save(output_path)
        print(f"✓ Sample prescription created: {output_path}")
        return output_path
    except Exception as e:
        print(f"⚠ Could not create sample prescription: {e}")
        return None


In [9]:
def display_results(medicines: List[MedicineInfo]):
    print("\n" + "="*80)
    print("PRESCRIPTION ANALYSIS RESULTS")
    print("="*80)
    if not medicines:
        print("❌ No medicines detected in the prescription")
        return
    for i, medicine in enumerate(medicines, 1):
        print(f"\n📋 MEDICINE {i}")
        print("-" * 40)
        print(f"💊 Medicine Name: {medicine.name}")
        print(f"📝 Description: {medicine.description}")
        print(f"🎯 Uses: {medicine.uses}")
        print(f"💉 Recommended Dosage: {medicine.dosage}")
        print(f"🎯 Confidence Score: {medicine.confidence:.2f}")
        print("-" * 40)


In [16]:
def main():
    print("\n🚀 Starting Prescription Medicine Extraction Pipeline")
    extractor = MedicineExtractor()
    sample_image = create_sample_prescription()

    if sample_image:
        medicines = extractor.extract_medicines_from_image(sample_image)
        display_results(medicines)
    else:
        print("⚠ No image to process. Please provide a prescription image.")

    print("\n" + "="*60)
    print("TO PROCESS YOUR OWN PRESCRIPTION IMAGE:")
    print("="*60)
    print("1. Upload your prescription image")
    print("2. Use: medicines = extractor.extract_medicines_from_image('your_image.jpg')")
    print("3. Use: display_results(medicines)")

    extractor.db.close()
    print("\n✅ Pipeline completed successfully!")

# To run directly:
main()


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.



🚀 Starting Prescription Medicine Extraction Pipeline
✓ Database created with 35 medicines
✓ OCR engines initialized
✓ Sample prescription created: sample_prescription.png

🔍 Processing image: sample_prescription.png
✓ Extracted 61 text segments
✓ Found 26 potential medicines
✓ Matched 5 medicines in database

PRESCRIPTION ANALYSIS RESULTS

📋 MEDICINE 1
----------------------------------------
💊 Medicine Name: Paracetamol
📝 Description: Analgesic and antipyretic
🎯 Uses: Fever, headache, mild pain
💉 Recommended Dosage: 500mg every 6 hours
🎯 Confidence Score: 0.96
----------------------------------------

📋 MEDICINE 2
----------------------------------------
💊 Medicine Name: Ibuprofen
📝 Description: Non-steroidal anti-inflammatory drug
🎯 Uses: Pain, inflammation, fever
💉 Recommended Dosage: 400mg every 8 hours
🎯 Confidence Score: 0.55
----------------------------------------

📋 MEDICINE 3
----------------------------------------
💊 Medicine Name: Omeprazole
📝 Description: Proton pump inhi

In [17]:
extractor = MedicineExtractor()


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


✓ Database created with 35 medicines
✓ OCR engines initialized


In [18]:
image_path = "pres1.png"
medicines = extractor.extract_medicines_from_image(image_path)
display_results(medicines)


🔍 Processing image: pres1.png
✓ Extracted 77 text segments
✓ Found 11 potential medicines
✓ Matched 1 medicines in database

PRESCRIPTION ANALYSIS RESULTS

📋 MEDICINE 1
----------------------------------------
💊 Medicine Name: Cetirizine
📝 Description: Antihistamine
🎯 Uses: Allergies, hay fever
💉 Recommended Dosage: 10mg once daily
🎯 Confidence Score: 0.66
----------------------------------------


In [27]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
print(pytesseract.get_tesseract_version())


5.5.0.20241111


In [1]:
extractor = MedicineExtractor()

NameError: name 'MedicineExtractor' is not defined